##Packages

In [37]:
using DataFrames
using JuMP, Gurobi
using Gadfly

##Initiate Model, Load Data and Define General Parameters

In [47]:
m = Model(solver=GurobiSolver())

iGenLoad = readtable("BatteryModel_GenLoad.csv",header=true)
iPrices = readtable("BatteryModel_Prices.csv",header=true)
iParameters = readtable("BatteryModel_Parameters.csv",header=true)
iScheduledLoadsData = readtable("SchedulableLoads_Parameters.csv", header=true) 
iThermalParameters = readtable("Thermal_Parameters.csv", header=true) 

T = length(iPrices[:pHour]) 
pMonth = iPrices[:pMonth][1:T]
pWeek = iPrices[:pWeek][1:T]
pHour = iPrices[:pHour][1:T]

pBuyEnergy = iPrices[:pBuy_Energy][1:T]
pSellEnergy = iPrices[:pSell_Energy][1:T]
pNonControllableLoad = iGenLoad[:pNonControllableLoad][1:T]

pNetworkPeakHour = 117
pNetworkCostperkW = 30
pDelta = 1 #duration of time period (hr)
pCCurtail = 0.25 #cost of curtailment - will depend on the scenario ($/KWH)
pGenAssets = [5 5 2.5 2.5] # this is where you specify the technology choices; will ultimately take this from dataframes
#genAssets = [pPVbank pBatt_NominalE pBatt_DischargeCapacity pBatt_ChargeCapacity]
#pGenAssets = [15 5 2.5 2.5]

#NON-CONTROLLABLE GEN PARAMETERS
pPV_Capacity = pGenAssets[1] #this is the total capacity of the PV panels 
pPV_Generation = pGenAssets[1]*iGenLoad[:pPV_Generation][1:T]
pOtherNonControllableGen = iGenLoad[:pOtherNonControllableGen][1:T]  # declaring non-PV based non-controllable generation
@defExpr(pTotalNonControllableGen[t=1:T], pPV_Generation[t]+pOtherNonControllableGen[t]) #total non-controllable generation = PV + other


#SCHEDULABLE LOADS PARAMETERS
pNumCycles = 170
pLoadTime = iScheduledLoadsData[:pLoadTime][1:pNumCycles]
pFinishTime = iScheduledLoadsData[:pFinishTime][1:pNumCycles]
pMaxLoad = 1
pTotal_SL_kWh = 1


#THERMAL PARAMETERS
# all values from "Using Residential Electric Loads for Fast Demand Response"
# Mathieu, Dyson, Callaway - 2012 ACEEE Summer Study
pOutdoorTemp = iGenLoad[:pOutdoorTemp][1:T]
pSetpoint = iGenLoad[:pSetPoint][1:T]
pDeadband = iThermalParameters[1,:pDeadband]
pCapacitance = iThermalParameters[1,:pCapacitance]              #heat capacity of building interior (kWh/C) 
pResistance = 4 #iThermalParameters[1,:pResistance]                #thermal resistance of between interior and exterior (C/kWh) 
pCOP = iThermalParameters[1,:pCOP]                              #COP of heat pump (eventually convert to piecewise linear)
pMaxPower = iThermalParameters[1,:pMaxPower]
pTempDevPenalty = iThermalParameters[1,:pTempDevPenalty]        #temperature deviation penalty ($/deg C)

pWH_AmbientTemp = 20
pWH_Setpoint = 50
pWH_Deadband = iThermalParameters[2,:pDeadband]
pWH_Capacitance = iThermalParameters[2,:pCapacitance]              #heat capacity of building interior (kWh/C) 
pWH_Resistance = iThermalParameters[2,:pResistance]                #thermal resistance of between interior and exterior (C/kWh) 
pWH_COP = iThermalParameters[2,:pCOP]                              #COP of heat pump (eventually convert to piecewise linear)
pWH_MaxPower = iThermalParameters[2,:pMaxPower]


#BATTERY PARAMETERS
pBattNominalE = pGenAssets[2]
pBattDischargeCapacity = pGenAssets[3] #installed battery discharge power capacity (KW)
pBattChargeCapacity = pGenAssets[4] #installed battery charge power capacity (KW)

pBattDischargeEff = iParameters[1,:pBatt_DischargeEff] #pBatt_DischargeEff Efficiency of battery discharge
pBattChargeEff = iParameters[1,:pBatt_ChargeEff] #pBattChargeEff Efficiency of battery charge
pBattCDeg = iParameters[1,:pBatt_CDeg] #pBattCDeg battery degradation cost ($) 
pBattInitialSOC = iParameters[1,:pBatt_InitialSOC] #pBatt_InitialSOC initial state of charge 
pBattSOCMax = iParameters[1,:pBatt_SOCMax] #pBatt_SOCMax maximum SOC
pBattSOCMin = iParameters[1,:pBatt_SOCMin] #pBatt_SOCMin minimum SOC
;

###Variable Declarations

In [48]:
#SCHEDULABLE LOADS
@defVar(m, 0<=vSL[t=1:T,1:pNumCycles]<=pMaxLoad)

#THERMAL LOADS
@defVar(m, sTempInt[t=1:T] >=0)                 # internal home temp (state variable)
@defVar(m, sExtLosses[t=1:T])                   # losses/gains from thermal leakage through building shell
@defVar(m, sIntGains[t=1:T])                    # internal gains
@defVar(m, 0 <= vPowerHP[t=1:T] <= pMaxPower)         # HVAC power draw (continuous, 5kW max) | allow negative values = cooling???
@defVar(m, 0 <= vPowerAC[t=1:T] <= pMaxPower)
@defVar(m, vTempLow[t=1:T]>=0)                                       # penalty for temp deviations
@defVar(m, vTempHigh[t=1:T]>=0) 

#WATER HEAETER
@defVar(m, sWH_TempInt[t=1:T] >=0)                 # internal tank temp (state variable)
@defVar(m, sWH_ExtLosses[t=1:T])                   # losses/gains from thermal leakage through building shell
@defVar(m, sWH_IntGains[t=1:T])                    # internal gains
@defVar(m, 0 <= vPowerWH[t=1:T] <= pWH_MaxPower)         # WH power draw 
@defVar(m, vWH_TempLow[t=1:T]>=0)                                       # penalty for temp deviations
@defVar(m, vWH_TempHigh[t=1:T]>=0)

#BATTERY
@defVar(m, pBattSOCMin <= vBattSOC[t=1:T] <= pBattSOCMax) # SOC of battery ----- SB!!! Maybe make this a state variable, not a decision variable.
@defVar(m, vBattSOH[t=1:T] >= 0) #State of health of battery (degradation costs)
@defVar(m, vBattCharge[t=1:T] >= 0) #battery charging power
@defVar(m, vBattDischarge[t=1:T] >= 0) #battery discharging power
@defVar(m, vBattCorD[t=1:T], Bin) #batt charging or discharging binary variable. 

#DEMAND BALANCE
@defVar(m, vPowerCurtail[t=1:T] >= 0) #curtailed power
@defVar(m, vPowerConsumed[t=1:T] >=0) # consumed power
@defVar(m, vPowerImportorExport[t=1:T], Bin)
@defVar(m, vPowerPurchased[t=1:T] >=0) # power ultimately purchased from utility
@defVar(m, sPowerExport[t=1:T] >= 0)
;

###Constraint Declarations

In [49]:
#SCHEDULABLE LOADS
for w = 1:pNumCycles
    @addConstraint(m, pTotal_SL_kWh==sum{vSL[t,w],t=pLoadTime[w]:pFinishTime[w]})
    @addConstraint(m, pTotal_SL_kWh==sum{vSL[t,w],t=1:T})
end 
@defExpr(vScheduledLoads[t=1:T], sum{vSL[t,w], w=1:pNumCycles})


#WATER HEATING LOADS
for t = 1
    @addConstraint(m, sWH_TempInt[t]==pWH_Setpoint)                         #starting temp
end
for t = 2:T
    @addConstraint(m,sWH_TempInt[t-1]+((sWH_ExtLosses[t-1]+sWH_IntGains[t-1])/pWH_Capacitance)==sWH_TempInt[t]) # temp evolution | temp(t) = temp(t-1) + (gains - losses)/heat capacity    
end
for t = 1:T
    @addConstraint(m, ((pWH_AmbientTemp - sWH_TempInt[t])/(pWH_Capacitance*pWH_Resistance))==sWH_ExtLosses[t])   #losses from thermal leakage
    @addConstraint(m, ((pWH_COP*vPowerWH[t])/pWH_Capacitance)==sWH_IntGains[t])                         #interanl temp gain from heat pump  
    @addConstraint(m, (pWH_Setpoint-pWH_Deadband)-vWH_TempLow[t]<=sWH_TempInt[t])
    @addConstraint(m, (pWH_Setpoint+pWH_Deadband)+vWH_TempHigh[t]>=sWH_TempInt[t])
end


#HEATING/COOLING LOADS
for t = 1
    @addConstraint(m, sTempInt[t]==pSetpoint[t])                         #starting temp
end
for t = 2:T
    @addConstraint(m,sTempInt[t-1]+((sExtLosses[t-1]+sIntGains[t-1])/pCapacitance)==sTempInt[t]) # temp evolution | temp(t) = temp(t-1) + (gains - losses)/heat capacity    
end
for t = 1:T
    @addConstraint(m, ((pOutdoorTemp[t] - sTempInt[t])/(pCapacitance*pResistance))==sExtLosses[t])   #losses from thermal leakage
    @addConstraint(m, ((pCOP*vPowerHP[t])/pCapacitance)-((2.5*vPowerAC[t])/pCapacitance)==sIntGains[t])                         #interanl temp gain from heat pump
    @addConstraint(m, (pSetpoint[t]-pDeadband)-vTempLow[t]<=sTempInt[t])
    @addConstraint(m, (pSetpoint[t]+pDeadband)+vTempHigh[t]>=sTempInt[t])
    
    @addConstraint(m, vPowerHP[t]+vPowerAC[t]<=pMaxPower)
end
@defExpr(vThermalLoad[t=1:T], vPowerHP[t]+vPowerAC[t]+vPowerWH[t])
@defExpr(vTotalTempDev[t=1:T],vTempHigh[t]+vTempLow[t]+vWH_TempHigh[t]+vWH_TempLow[t])

In [50]:
#BATTERY

#constraining state of charge to appropriate limits
@addConstraint(m, vBattSOC[1] == pBattInitialSOC)


# defining charging losses and efficiency
@defExpr(sBattDischargeLosses[t=1:T], pBattDischargeEff*vBattDischarge[t])
@defExpr(sBattChargeLosses[t=1:T], pBattChargeEff*vBattCharge[t])
@defExpr(sBattLosses[t=1:T], sBattChargeLosses[t] + sBattDischargeLosses[t])

@defExpr(sBattOutput[t=1:T], vBattDischarge[t] - sBattDischargeLosses[t])
@defExpr(sBattInput[t=1:T], vBattCharge[t] + sBattChargeLosses[t])

if pBattNominalE > 0
    for t=1:T
        @addConstraint(m, vBattCharge[t]<=pBattChargeCapacity)
        @addConstraint(m, vBattDischarge[t]<=pBattDischargeCapacity)
    end
else
    for t=1:T
        @addConstraint(m, vBattCharge[t]==0)
        @addConstraint(m, vBattDischarge[t]==0)
    end
end
    
if pBattNominalE > 0
    for t = 2:T  
        @addConstraint(m, vBattSOC[t-1] - (pDelta/pBattNominalE)*(sBattOutput[t-1] - sBattInput[t-1] + sBattLosses[t-1]) == vBattSOC[t])
    end 
else 
    for t = 2:T
        @addConstraint(m, vBattSOC[t] == vBattSOC[1])
    end 
end 

#define charge or discharge constraint
for t=1:T
    @addConstraint(m, vBattCharge[t] <= 1000000*vBattCorD[t])
    @addConstraint(m, vBattDischarge[t] <= 1000000*(1-vBattCorD[t]))
end


In [51]:
#DEMAND BALANCE

@defExpr(sPowerProduced[t=1:T], pTotalNonControllableGen[t] + vBattDischarge[t])

for t=1:T
    @addConstraint(m, pNonControllableLoad[t]+vScheduledLoads[t]+vThermalLoad[t]+vBattCharge[t]==vPowerConsumed[t])     
    @addConstraint(m, vPowerConsumed[t]+sPowerExport[t]==vPowerPurchased[t]+sPowerProduced[t])
    
    #cannot import and export at the same time
    @addConstraint(m, sPowerExport[t] <= 10000000*(1-vPowerImportorExport[t]))
    @addConstraint(m, vPowerPurchased[t] <= 10000000*(vPowerImportorExport[t]))
end

##Objective Function

In [52]:
@defExpr(EndUseLoads[t=1:T], pNonControllableLoad[t]+vScheduledLoads[t]+vThermalLoad[t])
@defExpr(NetworkCost, pNetworkCostperkW*vPowerPurchased[pNetworkPeakHour])
@defExpr(EnergyCost, sum{pBuyEnergy[t]*vPowerPurchased[t],t=1:T}+pTempDevPenalty*sum{vTotalTempDev[t],t=1:T})
@defExpr(TotalCost, EnergyCost+NetworkCost)
@defExpr(TotalRevenue, sum{pSellEnergy[t]*sPowerExport[t],t=1:T})
@defExpr(TotalPowerProvided[t=1:T],vPowerConsumed[t]+sPowerExport[t])
@setObjective(m, Min, TotalCost - TotalRevenue);
;

In [53]:
# Print outputs
status = solve(m)
status
println("Objective value: ", getObjectiveValue(m))

# Time records:
# 168 hours, flat prices: 0s
# 8760 hours, flat prices: 4s
# 8760 hours, RTP prices: 4s
# 8760 hours, RTP prices, many SL: 6s
# 8760 hours, RTP prices, many SL: 7s
# 8760, RTP, Battery, SLs, WH: 14s
# 8760, RTP, Battery, SLs, WH, HP/AC: 15s

Optimize a model with 175540 rows, 1690680 columns and 3417425 nonzeros
Coefficient statistics:
  Matrix range    [2e-02, 1e+07]
  Objective range [2e-05, 3e+01]
  Bounds range    [1e-01, 5e+00]
  RHS range       [4e-05, 1e+07]
Presolve removed 17533 rows and 17533 columns (presolve time = 5s) ...
Presolve removed 67167 rows and 67168 columns (presolve time = 10s) ...
Presolve removed 71933 rows and 1563088 columns
Presolve time: 13.44s
Presolved: 103607 rows, 127592 columns, 300552 nonzeros
Variable types: 113391 continuous, 14201 integer (14201 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.9353474e+02   2.814792e+05   0.000000e+00     21s
   52949    1.3272487e+03   0.000000e+00   0.000000e+00     22s

Root relaxation: objective 1.327249e+03, 52949 iterations, 1.62 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time



##Reporting

In [58]:
# Build output array
aTemp = hcat(pMonth[1:T],
        pWeek[1:T],
        pHour[1:T],
        pBuyEnergy[1:T],
        pSellEnergy[1:T],
        getValue(vPowerConsumed[1:T]),
        getValue(sPowerProduced[1:T]),
        getValue(vPowerPurchased[1:T]),
        getValue(sPowerExport[1:T]),
        getValue(vBattDischarge[1:T]),
        getValue(vBattCharge[1:T]),
        pSetpoint[1:T],
        pOutdoorTemp[1:T],
        getValue(sTempInt[1:T]),
        getValue(vBattSOC[1:T]),
        getValue(vBattSOH[1:T]),
        getValue(vPowerHP[1:T]),
        getValue(vScheduledLoads[1:T]),
        getValue(vPowerWH[1:T]),
        getValue(vPowerAC[1:T]),
        pNonControllableLoad[1:T],
        getValue(EndUseLoads[1:T])
        )

aTemp = convert(Array, aTemp)

dfUsage = convert(DataFrame, aTemp)

rename!(dfUsage, {:x1=>:Month,
            :x2=>:Week, 
            :x3=>:Hour, 
            :x4=>:Buy_Energy, 
            :x5=>:Sell_Energy, 
            :x6=>:Power_Consumed,
            :x7=>:Power_Produced,
            :x8=>:Power_Purchased,
            :x9=>:Power_Export,
            :x10=>:Batt_Discharge,
            :x11=>:Batt_Charge,
            :x12=>:Set_point,
            :x13=>:Outdoor_temp,
            :x14=>:Indoor_temp,
            :x15=>:Battery_SOC,
            :x16=>:Battery_SOH,
            :x17=>:HP_kW,
            :x18=>:Scheduled_Loads_kW,
            :x19=>:WH_kW,
            :x20=>:AC_kW,
            :x21=>:NonControllableLoads_kW,
            :x22=>:EndUseLoads_kW
            })

writetable("outputs.csv",dfUsage)

bTemp = convert(Array,getValue(vSL[1:T,1:52]))
dfTemp = convert(DataFrame, bTemp)
writetable("SL outputs.csv",dfTemp)

In [38]:
# Check if file is open and close it so it can be updated

    #f = open("outputs2.csv")
    #isopen(f)
aTemp

8760x17 Array{Float64,2}:
  1.0   1.0     1.0  0.04565  0.1  …   7.0  20.0     0.0      0.5  0.0
  1.0   1.0     2.0  0.03972  0.1      7.0  18.375   3.49107  0.1  0.0
  1.0   1.0     3.0  0.0454   0.1      7.0  19.5714  0.0      0.6  0.0
  1.0   1.0     4.0  0.04039  0.1      6.5  18.0     1.91667  0.1  0.0
  1.0   1.0     5.0  0.04189  0.1      6.0  18.0     4.66667  0.6  0.0
  1.0   1.0     6.0  0.03961  0.1  …   6.0  20.0     2.33333  0.1  0.0
  1.0   1.0     7.0  0.03737  0.1      4.0  20.0     2.66667  0.1  0.0
  1.0   1.0     8.0  0.03577  0.1      5.0  20.0     2.5      0.1  0.0
  1.0   1.0     9.0  0.0347   0.1      6.0  20.0     2.33333  0.1  0.0
  1.0   1.0    10.0  0.03303  0.1      4.0  20.0     5.0      0.5  0.0
  1.0   1.0    11.0  0.03801  0.1  …   4.0  21.75    0.625    1.0  0.0
  1.0   1.0    12.0  0.03601  0.1      3.0  20.0     5.0      0.5  0.0
  1.0   1.0    13.0  0.04303  0.1      3.0  21.625   0.9375   1.0  0.0
  ⋮                                ⋱               

In [63]:
plot(layer(x=1:24,y=getValue(sPowerProduced[1:T]) ,Geom.point, Geom.line, Theme(default_color=colorant"yellow")),
layer(x=1:24,y=getValue(vPowerConsumed[1:T]) ,Geom.point, Geom.line, Theme(default_color=colorant"blue")),
layer(x=1:24,y=getValue(sPowerExport[1:T]) ,Geom.point, Geom.line, Theme(default_color=colorant"orange")),
layer(x=1:24,y=getValue(vBattCharge[1:T]) ,Geom.point, Geom.line, Theme(default_color=colorant"green")),
layer(x=1:24,y=getValue(vBattDischarge[1:T]) ,Geom.point, Geom.line, Theme(default_color=colorant"red")))


ErrorException: The following aesthetics are required by Geom.point to be of equal length: x, y


In [1]:
plot(layer(x=1:24,y=getValue(sPowerProduced[1:24]) ,Geom.point, Geom.line, Theme(default_color=colorant"yellow")),
layer(x=1:24,y=getValue(TotalPowerProvided[1:24]) ,Geom.point, Geom.line, Theme(default_color=colorant"blue")),
layer(x=1:24,y=getValue(vPowerHVAC[1:24]) ,Geom.point, Geom.line, Theme(default_color=colorant"orange")))


LoadError: @colorant_str not defined
while loading In[1], in expression starting on line 4

LoadError: syntax: extra token "status" after end of expression
while loading In[49], in expression starting on line 1

To do in Gadfly:
    - plot on two y-axes
    - extend width 